In [1]:
import os
import openai
from openai import OpenAI
import requests
from bs4 import BeautifulSoup
import nltk
import html2text
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize, sent_tokenize 

## Set OpenAI ApiKey

API_KEY = ''




In [2]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\nivsi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\nivsi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## Idea:
 - Is this all in plaintext or is there some structure here?
 - Could we have multiple connectors? So the worst is plaintext, but if you can augment it with K9DB or RuleKeeper or something, it gets better?


Question Gen
- https://www.asd5.org/cms/lib4/WA01001311/Centricity/Domain/145/Costas.pdf

In [3]:
client = OpenAI(api_key=API_KEY)
def ask_gpt(systemprompt, userprompt):
    completion = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": systemprompt},
        {"role": "user", "content": userprompt}
    ]
    )

    return completion.choices[0].message

# Use this for now -- Gradescope policy, for instance, is FAR TOO LONG for GPT-3 completions. Need to think
# of a way to split it up.
def get_hardcoded_policy(url):
    return '''Gradescope collects the following types of data and shares them with the corresponding entities:

- Personal information: This may include names, email addresses, and other identifying details provided by users. It is shared with Gradescope's internal team, employees, and authorized service providers who need access to the information to perform their duties and improve the service. For example, the internal team may use personal information to verify user identities, provide customer support, and communicate important updates. Authorized service providers may include hosting providers, data storage providers, and analytics platforms who assist Gradescope in delivering and improving the service.
- Usage data: This includes information about how users interact with the platform, such as the courses they enroll in, assignments they submit, and grades they receive. It is shared with Gradescope's internal team, employees, and authorized service providers who analyze the data to enhance the platform's features and functionality. This analysis helps Gradescope understand user preferences, identify areas for improvement, and make data-driven decisions to provide a better user experience. The internal team may also use usage data to monitor system performance, troubleshoot issues, and ensure the proper functioning of the platform.

Gradescope takes appropriate measures to safeguard the collected data and respects user privacy in accordance with its privacy policy. These measures include implementing technical and organizational security measures to protect against unauthorized access, maintaining strict data confidentiality obligations for employees and service providers, and regularly monitoring and updating security practices to align with industry standards. It is important for users to review and understand the privacy policy to be aware of their rights and choices regarding their data.
Gradescope does not sell or share personal information with third parties for marketing purposes. The collected data is used solely for the purposes stated in the privacy policy and to provide and improve the Gradescope service. Users can trust that their data is handled with care and in compliance with applicable laws and regulations.'''

def get_policy_from_web(url):
    response = requests.get(url)

    # Check if the request was successful
    if response.status_code == 200:
        # Parse the HTML content of the page using BeautifulSoup
        soup = BeautifulSoup(response.text, 'html.parser')
        text_with_html = soup.get_text()
        text_without_html = html2text.html2text(text_with_html)
        return text_without_html
    else:
        print("Failed to retrieve the webpage. Status code:", response.status_code)


# VERY BASIC, from https://www.geeksforgeeks.org/python-text-summarizer/
def summarizer(text):
    
    # Tokenizing the text 
    stopWords = set(stopwords.words("english")) 
    words = word_tokenize(text) 
    
    # Creating a frequency table to keep the score of each word 
    
    freqTable = dict() 
    for word in words: 
        word = word.lower() 
        if word in stopWords: 
            continue
        if word in freqTable: 
            freqTable[word] += 1
        else: 
            freqTable[word] = 1
    
    # Creating a dictionary to keep the score of each sentence 
    sentences = sent_tokenize(text) 
    sentenceValue = dict() 
    
    for sentence in sentences: 
        for word, freq in freqTable.items(): 
            if word in sentence.lower(): 
                if sentence in sentenceValue: 
                    sentenceValue[sentence] += freq 
                else: 
                    sentenceValue[sentence] = freq 
    
    sumValues = 0
    for sentence in sentenceValue: 
        sumValues += sentenceValue[sentence] 
    
    # Average value of a sentence from the original text 
    
    average = int(sumValues / len(sentenceValue)) 
    
    # Storing sentences into our summary. 
    summary = '' 
    for sentence in sentences: 
        if (sentence in sentenceValue) and (sentenceValue[sentence] > (1.2 * average)): 
            summary += " " + sentence 
    return summary

## Costas Levels of Questioning

https://www.asd5.org/cms/lib4/WA01001311/Centricity/Domain/145/Costas.pdf

- LEVEL	1: Remember Define, Show Understanding
- LEVEL 2: Use Understanding, Examine, Create
- LEVEL 3: Decide, Show Understanding, Supportive Evidence


In [4]:

def get_costas_level_prompt(level):
    if level == 1:
        return ""
    elif level == 2:
        return ""
    elif level == 3:
        return ""
    return ""

def gen_question(context, level):
    return ""

## Flow 1
- Ask Question (Gen? Mockup? C.R.O.W.D.? Costas?)
- Student Answers
- Extract student answer, compare with privacy policy


In [5]:
# Will abstract out to various connectors
def get_system_prompt(policy_url):

    plaintext_policy = get_policy_from_web(policy_url)

    def rough_num_words(s):
        return s.count(" ") + 1
    
    c = 0
    ## HAck bc of OpenAI Limits
    while rough_num_words(plaintext_policy) > 3000 or c > 2:
        print("SUMMARIZE!")
        c += 1
        plaintext_policy = summarizer(plaintext_policy)


    PLAINTEXT_SYSTEM_PROMPT = ''' You are a tutor helping determine whether users correctly understand the privacy policy below:
        [PRIVACY POLICY]
        {p}

        For this policy, the student was asked a [QUESTION] and responded with an [ANSWER]. 
        Determine whether [ANSWER] answers [QUESTION], whether it was correct or incorrect.
        {{
            'Relevant' : [True if [ANSWER] answers [QUESTION], False otherwise],
            'Correct' : [True if the answer is correct, False if incorrect],
            'Explanation' : [A 2 sentence explanation of why the answer was correct or incorrect],
            'Confidence' : [Your confidence in this decision on a scale of 0 to 1, with 0 being not confident and 1 being completely certain.]
        }}

    '''.format(p = plaintext_policy)
    return PLAINTEXT_SYSTEM_PROMPT


def get_user_prompt(question, answer):
    return ''' 

        The student was asked the following question:
        
        [QUESTION] {q}

        and gave the following answer:
        [ANSWER] {a}
    '''.format(q = question, a = answer)


In [6]:
def check_qa(q, a, pp):
    system_prompt = get_system_prompt(policy_url=pp)
    user_prompt = get_user_prompt(question = q, answer = a)
    return ask_gpt(systemprompt=system_prompt, userprompt= user_prompt)

In [7]:
# Lets test it out

policy = 'https://www.gradescope.com/privacy'
question = 'With whom does Gradescope share your personal information?'
answer = "Gradescope only shares my personal information with course instructors"

print(check_qa(q = question, a = answer, pp = policy))



SUMMARIZE!
SUMMARIZE!
ChatCompletionMessage(content="{\n    'Relevant': True,\n    'Correct': False,\n    'Explanation': 'The answer is incorrect. The privacy policy does not specifically mention sharing personal information with course instructors.',\n    'Confidence': 0.9\n}", role='assistant', function_call=None)


## Flow 2:
- Student can ask questions about the Privacy Explorer
- Explorer uses GPT to answer.


In [8]:
# Will abstract out to various connectors
def flow_2_system_prompt(policy_url):


    plaintext_policy = get_policy_from_web(policy_url)

    def rough_num_words(s):
        return s.count(" ") + 1
    
    c = 0
    ## Hack bc of OpenAI Limits
    while rough_num_words(plaintext_policy) > 3000 or c > 2:
        print("SUMMARIZE!")
        c += 1
        plaintext_policy = summarizer(plaintext_policy)


    PLAINTEXT_SYSTEM_PROMPT = ''' You are a tutor helping answer user questions about the privacy policy below:
        [PRIVACY POLICY]
        {p}


    '''.format(p = plaintext_policy)
    return PLAINTEXT_SYSTEM_PROMPT


def flow_2_user_prompt(question):
    return ''' 
        For this [PRIVACY POLICY], the student asked the following [QUESTION] and responded with an [ANSWER]. 
        
        [QUESTION] {q}

        Answer this question using only information from [PRIVACY POLICY]:
        {{
            'Answer' : [A short answer to the question],
            'Confidence' : [Your confidence in this decision on a scale of 0 to 1, with 0 being not confident and 1 being completely certain.]
        }}
    '''.format(q = question, a = answer)


In [9]:
def gen_a(q, pp):
    system_prompt = flow_2_system_prompt(policy_url=pp)
    user_prompt = flow_2_user_prompt(question = q)
    return ask_gpt(systemprompt=system_prompt, userprompt= user_prompt)

print(gen_a(q = question,  pp = policy))

SUMMARIZE!
SUMMARIZE!
ChatCompletionMessage(content="{\n            'Answer': 'Gradescope shares personal information with third-party service providers, such as language translation service providers, who have contractual agreements to handle and secure the data with the same level of protection as Turnitin.',\n            'Confidence': 0.9\n        }", role='assistant', function_call=None)
